## Importing Libraries

In [11]:
import cv2
import csv
import collections
import numpy as np
from tracker import *

## Model Parameters

In [12]:
# YOLO expects an input size of 320
input_size = 320
# All the classes available in the COCO Dataset
classesFile = "coco.names"
classNames = open(classesFile).read().strip().split('\n')
print(classNames)
print("Coco has a number of classes of : ",len(classNames)-1)

# We can specify the classes we want yolo to predict
required_class_index = [2, 3,  7]

detected_classNames = []

# Detection confidence threshold for tracking  and NMS threshold for bounding boxes
confThreshold = 0.2
nmsThreshold = 0.2

# Text color and thickness
font_color = (0, 0, 255)
font_size = 0.5
font_thickness = 2

# Middle cross line position for counting
middle_line_position = 225
up_line_position = middle_line_position - 15
down_line_position = middle_line_position + 15


# Lists for vehicle count information storage
temp_up_list = []
temp_down_list = []
up_list = [0, 0, 0, 0]
down_list = [0, 0, 0, 0]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', '>>>>>>> c91dd7980f88de76c049a0b501c7530b77180c8f']
Coco has a number of classes of :  80


In [13]:
# Function to Draw Bounding boxes and add text for each object found
def postProcess(outputs, img):
    #  detected_classNames = all detected classes in a frame
    global detected_classNames
    height, width = img.shape[:2]
    boxes = []
    classIds = []
    confidence_scores = []
    detection = []
    for output in outputs:
        for det in output:
            # We iterate through all the objects detected in each frame and get all the scores
            scores = det[5:]
            # We take the class and the confidence score of each detected object
            classId = np.argmax(scores)
            confidence = scores[classId]
            # if it's in our list we add it to the list of boxes
            if classId in required_class_index:
                if confidence > confThreshold:     
                    w, h = int(det[2]*width), int(det[3]*height)
                    x, y = int((det[0]*width)-w/2), int((det[1]*height)-h/2)
                    boxes.append([x, y, w, h])
                    classIds.append(classId)
                    confidence_scores.append(float(confidence))

    # Apply Non-Max Suppression in the case of multiple bounding boxes in a single object
    indices = cv2.dnn.NMSBoxes(
        boxes, confidence_scores, confThreshold, nmsThreshold)
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3]
            # print(x,y,w,h)

            color = [int(c) for c in colors[classIds[i]]]
            name = classNames[classIds[i]]
            detected_classNames.append(name)
            # Draw classname and confidence score
            cv2.putText(img, f'{name.upper()} {int(confidence_scores[i]*100)}%',
                        (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            # Draw bounding rectangle
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            detection.append(
                [x, y, w, h, required_class_index.index(classIds[i])])

    # Update the tracker for each object
    boxes_ids = tracker.update(detection)
    for box_id in boxes_ids:
        count_vehicle(box_id, img)

In [14]:
# Function to count the vehicle passing through the middle line
def count_vehicle(box_id, img):

    x, y, w, h, id, index = box_id

    # Find the center of the rectangle for detection
    center = find_center(x, y, w, h)
    ix, iy = center

    # Find the current position of the vehicle
    if (iy > up_line_position) and (iy < middle_line_position):

        if id not in temp_up_list:
            temp_up_list.append(id)

    elif iy < down_line_position and iy > middle_line_position:
        if id not in temp_down_list:
            temp_down_list.append(id)

    elif iy < up_line_position:
        if id in temp_down_list:
            temp_down_list.remove(id)
            up_list[index] = up_list[index]+1

    elif iy > down_line_position:
        if id in temp_up_list:
            temp_up_list.remove(id)
            down_list[index] = down_list[index] + 1

    # Draw circle in the middle of the rectangle
    cv2.circle(img, center, 2, (0, 0, 255), -1)  

def find_center(x, y, w, h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx, cy

## Vehicle Counting from an Image

In [18]:
image = 'img.jpg'
detected_classNames=[]
img = cv2.imread(image)
blob = cv2.dnn.blobFromImage(
        img, 1 / 255, (input_size, input_size), [0, 0, 0], 1, crop=False)

tracker = EuclideanDistTracker()

modelConfiguration = 'yolov3.cfg'
modelWeigheights = 'yolov3.weights'
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeigheights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Define random colour for each class
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classNames), 3), dtype='uint8')

net.setInput(blob)
layersNames = net.getLayerNames()
outputNames = [(layersNames[i - 1]) for i in net.getUnconnectedOutLayers()]
outputs = net.forward(outputNames)
postProcess(outputs, img)
# count the frequency of detected classes
frequency = collections.Counter(detected_classNames)
print(frequency)
# Draw counting texts in the frame
cv2.putText(img, "Car:        "+str(frequency['car']), (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.putText(img, "Motorbike:  "+str(frequency['motorbike']), (20, 60),
            cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)
cv2.putText(img, "Truck:      "+str(frequency['truck']), (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, font_size, font_color, font_thickness)

cv2.imshow("image", img)

cv2.waitKey(0)

# save the data to a csv file
with open("static-data.csv", 'a') as f1:
    cwriter = csv.writer(f1)
    cwriter.writerow([image, frequency['car'], frequency['motorbike'],
                         frequency['truck']])
f1.close()


Counter({'car': 26, 'truck': 3})
